In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib.request import urlopen as ureq
import re

In [3]:
df = pd.DataFrame(columns =['price', 'locali', 'superficie', 'bagni','piano'])

In [25]:
links=[]
for i in range(1,10):
    content = requests.get("https://www.immobiliare.it/vendita-case/roma/?criterio=rilevanza&pag="+str(i))
    soup = BeautifulSoup(content.text, 'html.parser')
    for link in soup.find_all('a', href=True):
        url = link['href']
        if url.startswith('https://www.immobiliare.it/') and url.endswith('.html'):
            print(url)

https://www.immobiliare.it/53131931-Vendita-Bilocale-viale-Italo-Calvino-Roma.html
https://www.immobiliare.it/69866648-Vendita-Quadrilocale-via-Mario-Ridolfi-32-Roma.html
https://www.immobiliare.it/68373749-Vendita-Appartamento-via-dei-Gandolfi-Roma.html
https://www.immobiliare.it/68083741-Vendita-Villa-via-Casalmonferrato-Roma.html
https://www.immobiliare.it/65062100-Vendita-Appartamento-piazza-Cavour-Roma.html
https://www.immobiliare.it/63857680-Vendita-Quadrilocale-viale-della-Grande-Muraglia-Roma.html
https://www.immobiliare.it/67959361-Vendita-Appartamento-piazza-Del-Fante-Roma.html
https://www.immobiliare.it/67012733-Vendita-Attico-Mansarda-via-della-Frezza-Roma.html
https://www.immobiliare.it/61733354-Vendita-Appartamento-via-Dandolo-Roma.html
https://www.immobiliare.it/70043828-Vendita-Appartamento-via-del-Calice-Roma.html
https://www.immobiliare.it/65779135-Vendita-Trilocale-via-dei-Banchi-Vecchi-Roma.html
https://www.immobiliare.it/64401288-Vendita-Villa-via-Cristoforo-Sabbad

https://www.immobiliare.it/63420032-Vendita-Attico-Mansarda-via-dei-Baullari-Roma.html
https://www.immobiliare.it/70376212-Vendita-Trilocale-via-Francesco-Gentile-Roma.html
https://www.immobiliare.it/69936424-Vendita-Attico-Mansarda-piazzale-Clodio-12-Roma.html
https://www.immobiliare.it/69728540-Vendita-Villa-via-di-Mezzocammino-Roma.html
https://www.immobiliare.it/69483980-Vendita-Quadrilocale-via-Dacia-22-Roma.html
https://www.immobiliare.it/66537679-Vendita-Quadrilocale-via-Francesco-Crispi-Roma.html
https://www.immobiliare.it/66093203-Vendita-Attico-Mansarda-piazzale-delle-Medaglie-Roma.html
https://www.immobiliare.it/70266408-Vendita-Appartamento-via-Dei-Monti-Parioli-Roma.html
https://www.immobiliare.it/69592474-Vendita-Appartamento-via-Della-Farnesina-Roma.html
https://www.immobiliare.it/70105452-Vendita-Quadrilocale-via-Virgilio-Melandri-30-Roma.html
https://www.immobiliare.it/68926129-Vendita-Appartamento-via-Edoardo-Jenner-163-Roma.html
https://www.immobiliare.it/67646595-Ve

https://www.immobiliare.it/70178590-Vendita-Appartamento-via-di-Monte-del-Gallo-6-Roma.html
https://www.immobiliare.it/66028417-Vendita-Appartamento-via-Nizza-Roma.html
https://www.immobiliare.it/69259598-Vendita-Appartamento-via-Archimede-Roma.html
https://www.immobiliare.it/58827600-Vendita-Villa-via-Cristoforo-Sabbadino-151D-Roma.html
https://www.immobiliare.it/69726630-Vendita-Quadrilocale-via-della-Maglianella-141-Roma.html
https://www.immobiliare.it/65613957-Vendita-Villa-via-Tommaso-Traetta-Roma.html
https://www.immobiliare.it/70180130-Vendita-Quadrilocale-piazza-di-Villa-Fiorelli-Roma.html
https://www.immobiliare.it/67995033-Vendita-Quadrilocale-viale-della-Tecnica-Roma.html
https://www.immobiliare.it/60692312-Vendita-Appartamento-via-Sardegna-Roma.html
https://www.immobiliare.it/70266422-Vendita-Quadrilocale-via-Livio-Pentimalli-Roma.html
https://www.immobiliare.it/64398844-Vendita-Villa-via-Cristoforo-Sabbadino-151D-Roma.html
https://www.immobiliare.it/64259182-Vendita-Appart

In [8]:
# Opening and picking up the content
PageLink = [Pagelink for Pagelink in Urls]
uPage = ureq(PageLink)
PageHtml = uPage.read()
uPage.close()

AttributeError: 'list' object has no attribute 'timeout'